## Including Fees in transaction configuration
In this example we demonstrate the configuration of transaction types that take fees into consideration. We have included an example of adding a non-capitalised fee (it is not considered in the cost basis of the resultant holding so we have to create a carry movement to account for it).

## Imports


In [ ]:

# Updated for LUSID SDK v2
from finbourne_sdk_utils import ApiClientFactory
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame

from finbourne_portfolio import models as portfolio_models
from finbourne_portfolio.api import (
    TransactionPortfoliosApi,
    PropertyDefinitionsApi,
    TransactionConfigurationApi,
    ApplicationMetadataApi,
    RelationshipsApi
)
from finbourne_instrument.api import InstrumentsApi

import pandas as pd
import json
import logging

# Use secrets file or environment variables
# api_factory = ApiClientFactory(config_path="secrets.json")  # Uncomment to use secrets file
api_factory = ApiClientFactory()

# Build API clients
instruments_api = api_factory.build(InstrumentsApi)
transaction_portfolios_api = api_factory.build(TransactionPortfoliosApi)
property_definition_api = api_factory.build(PropertyDefinitionsApi)
transaction_config_api = api_factory.build(TransactionConfigurationApi)
relationships_api = api_factory.build(RelationshipsApi)
application_metadata_api = api_factory.build(ApplicationMetadataApi)

# Get and display LUSID version
api_status = pd.DataFrame(application_metadata_api.get_lusid_versions().to_dict())
display(api_status)


In [ ]:
instruments_api = api_factory.build(InstrumentsApi)
transaction_portfolios_api = api_factory.build(TransactionPortfoliosApi)
property_definition_api = api_factory.build(PropertyDefinitionsApi)
transaction_config_api = api_factory.build(TransactionConfigurationApi)

In [ ]:
#setting up variables

scope = "FeesTesting"
code = "FeesTestV1"
print(f"'{scope}/{code}' scope and code created.")

custom_transaction_type = "TestType"
custom_side = "TestSide"

## 1. Create instruments, portfolio and transactions to work with

### 1.0 Load Transaction Data

In [ ]:
#reading transactions file with a funds in transaction and a buy of BP
#using example source file

transactions_df = pd.read_csv("wildcards-commission-txns.csv", keep_default_na = False)
transactions_df.index += 1
display(transactions_df)

### 1.1 Load Instruments

#### Loaded from df file

In [ ]:

# Updated for LUSID SDK v2
from finbourne_sdk_utils import ApiClientFactory
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame

from finbourne_portfolio import models as portfolio_models
from finbourne_portfolio.api import (
    TransactionPortfoliosApi,
    PropertyDefinitionsApi,
    TransactionConfigurationApi,
    ApplicationMetadataApi,
    RelationshipsApi
)
from finbourne_instrument.api import InstrumentsApi

import pandas as pd
import json
import logging

# Use secrets file or environment variables
# api_factory = ApiClientFactory(config_path="secrets.json")  # Uncomment to use secrets file
api_factory = ApiClientFactory()

# Build API clients
instruments_api = api_factory.build(InstrumentsApi)
transaction_portfolios_api = api_factory.build(TransactionPortfoliosApi)
property_definition_api = api_factory.build(PropertyDefinitionsApi)
transaction_config_api = api_factory.build(TransactionConfigurationApi)
relationships_api = api_factory.build(RelationshipsApi)
application_metadata_api = api_factory.build(ApplicationMetadataApi)

# Get and display LUSID version
api_status = pd.DataFrame(application_metadata_api.get_lusid_versions().to_dict())
display(api_status)


### 1.2 Create Portfolio

In [ ]:

portfolio_definition=portfolio_models.CreateTransactionPortfolioRequest(
    display_name="Fees Test Portfolio",
    code = code,
    base_currency = "GBP",
    created="2020-01-01T00:00:00Z",
    instrument_scopes = [f"{scope}{code}"],
)

try:
    create_portfolio_response=transaction_portfolios_api.create_portfolio(
        scope = scope,
        create_transaction_portfolio_request = portfolio_definition
    )
    print(f"Portfolio with display name '{create_portfolio_response.display_name}' created effective {str(create_portfolio_response.created)}")
except ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])

## 2. Create Transaction Properties for fees

#### In this case we use derived properties

In [ ]:

property_definition = portfolio_models.CreateDerivedPropertyDefinitionRequest(
    domain="Transaction",  
    scope="Fees",  
    code="NonCapFee1",  
    data_type_id=portfolio_models.ResourceId(
        scope="system",
        code="number"  
    ),
    derivation_formula="round(Units * TransactionPrice.Price * 0.1, 0.1)",
    display_name="Non-Cap Fee",  
    description="Derived property for non-capitalised fees",  
    is_filterable=False
)

response = property_definition_api.create_derived_property_definition(
    create_derived_property_definition_request=property_definition
)
print(f"Property definition created with key: {response.key}")

## 3. Create a Fees Side that uses the fee property

#### (Once functional, this can leverage the wildcard capability to access multiple fees in one side)

In [ ]:
#creating a side for non-capitalised fees

side_definition = portfolio_models.SideDefinitionRequest(
    security = "Txn:LusidInstrumentId",
    currency = "Txn:TradeCurrency",
    rate = "Txn:TradeToPortfolioRate",
    units = "Transaction/Fees/NonCapFee1",
    amount = "Transaction/Fees/NonCapFee1",
)

response = transaction_config_api.set_side_definition(
    # Specify the name of the custom side
    side = custom_side,
    side_definition_request = side_definition
)
display(response)

## 4. Create a "Buy"-style transaction type that includes a carry movement with the Fee Side

In [ ]:
#Create new transaction type

transaction_type_definition = portfolio_models.TransactionTypeRequest(
    aliases = [
        portfolio_models.TransactionTypeAlias(          
            type = custom_transaction_type,
            description = "Purchase with cash balance reduced by Non-Cap Fees",
            transaction_class = "Fees",
            transaction_roles = "LongLonger"
        )
    ],
    movements = [
        # Replicate the first movement from the built-in Buy transaction type
        portfolio_models.TransactionTypeMovement(
            movement_types = "StockMovement",
            side = "Side1",
            direction = 1,
            name = "Increase instrument holding by the number of units in the standard way",
        ),
        # Replicate the second movement from the built-in Buy transaction type
        portfolio_models.TransactionTypeMovement(
            movement_types = "CashCommitment",
            side = "Side2",
            direction = -1,
            name = "Decrease cash balance by total cost in the standard way",
        ),
        # Create a third movement that uses the custom side
        portfolio_models.TransactionTypeMovement(            
            movement_types = "Carry",
            direction = -1,
            side = custom_side,
            name = "Additionally decrease cash balance by commission",
            properties = {},
        ),
    ],
    #Update total consideration calculation to include non ca[pitalised fees
    calculations = [
         portfolio_models.TransactionTypeCalculation(
              type = "DeriveTotalConsideration",
              formula = "Txn:GrossConsideration + Properties[Transaction/Fees/NonCapFee1]"
         )
    ]
)


response = transaction_config_api.set_transaction_type(
    source = f"{code}",
    type = custom_transaction_type,
    transaction_type_request = transaction_type_definition
)
display(transaction_config_api.get_transaction_type(source = f"{code}", type = custom_transaction_type))


## 5. Create Transactions

In [ ]:
#Create transactions with this new type using the transactions data from previously 

transactions = []

for index, txn in transactions_df.iterrows():

    if txn["asset"] == "Cash":
        identifiers = {"Instrument/default/Currency": txn["currency"]}
        transaction_type = "FundsIn"
    else:
        identifiers = {"Instrument/default/Figi": txn["figi"]}
        transaction_type = custom_transaction_type

    transactions.append(
        portfolio_models.TransactionRequest(
            transaction_id = str(txn["txn_id"]),
            type = transaction_type,
            instrument_identifiers = identifiers,
            transaction_date = txn["trade_date"],
            settlement_date = txn["settle_date"],
            units = txn["units"],
            transaction_price = portfolio_models.TransactionPrice(price = txn["price"], type="Price"),
            total_consideration = portfolio_models.CurrencyAndAmount(
                amount = 0,
                currency = txn["currency"],
            ),
            properties = {},
            source = f"{code}"
        )
    )

upsert_transactions_response = transaction_portfolios_api.upsert_transactions(
    scope = scope, 
    code = code, 
    transaction_request = transactions
)

display(f"Transactions loaded at {str(upsert_transactions_response.version.as_at_date)}")
display(upsert_transactions_response)

## Look at resultant holdings

In [ ]:

# Updated for LUSID SDK v2
from finbourne_sdk_utils import ApiClientFactory
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame

from finbourne_portfolio import models as portfolio_models
from finbourne_portfolio.api import (
    TransactionPortfoliosApi,
    PropertyDefinitionsApi,
    TransactionConfigurationApi,
    ApplicationMetadataApi,
    RelationshipsApi
)
from finbourne_instrument.api import InstrumentsApi

import pandas as pd
import json
import logging

# Use secrets file or environment variables
# api_factory = ApiClientFactory(config_path="secrets.json")  # Uncomment to use secrets file
api_factory = ApiClientFactory()

# Build API clients
instruments_api = api_factory.build(InstrumentsApi)
transaction_portfolios_api = api_factory.build(TransactionPortfoliosApi)
property_definition_api = api_factory.build(PropertyDefinitionsApi)
transaction_config_api = api_factory.build(TransactionConfigurationApi)
relationships_api = api_factory.build(RelationshipsApi)
application_metadata_api = api_factory.build(ApplicationMetadataApi)

# Get and display LUSID version
api_status = pd.DataFrame(application_metadata_api.get_lusid_versions().to_dict())
display(api_status)
